## 反转效应（四）

In [ ]:
#===================================================================================================
# 反转效应（四）
#导包操作
from datetime import timedelta, date
import pandas as pd
#===================================================================================================
#===================================================================================================
#初始化函数，设置初始条件
def init(context):
    context.n = 25 
    run_monthly(trade,date_rule=-1)
    get_iwencai('沪深300,未停牌,上市时间超过2年')
#===================================================================================================
#===================================================================================================
#止损止盈函数
def handle_bar(context,bar_dict):
    #获取账户持仓信息
    holdstock = list(context.portfolio.stock_account.positions.keys()) 
    if len(holdstock) > 0:
        num = -0.1
        for stock in holdstock:
            close = history(stock,['close'],1,'1d').values
            if close/context.portfolio.positions[stock].last_price -1 <= num:
                order_target(stock,0)
                log.info('股票{}已止损'.format(stock))
#===================================================================================================
#===================================================================================================
#选股函数
def stocks_fz1(context,bar_dict):
    df = {'security': [], '30zf': []}
    stocks=context.iwencai_securities
    for symbol in stocks:
        df['security'].append(symbol)
    for i in range(len(df['security'])):
        quote = history(df['security'][i], ['quote_rate'], 30, '1d', True, fq='pre')
        AMP30 = quote.values[:].sum()
        df['30zf'].append(AMP30)
    df = pd.DataFrame(df).sort_values(by ='30zf', ascending=True)
    context.fz = df['security'][:50]
    return context.fz
def stocks_fz2(context,bar_dict):
    df = {'security': [], '30cje': []}
    security=stocks_fz1(context,bar_dict)
    #该策略选股函数中，我们通过一个函数的结果，导到第二个函数中去运行，可以达到多步骤选股
    stock=security
    for symbol in stock:
        df['security'].append(symbol)
    for i in range(len(df['security'])):
        quote = history(df['security'][i], ['turnover'], 30, '1d', True, fq='pre')
        AMP30 = quote.values[:].sum()
        df['30cje'].append(AMP30)
    df = pd.DataFrame(df).sort_values(by ='30cje', ascending=True)
    context.cje = df['security'][:25]
    return context.cje
#===================================================================================================
#===================================================================================================
#交易函数
def trade(context, bar_dict):
    fz_list = stocks_fz2(context,bar_dict)
    stock_list = list(set(fz_list))
    if len(list(context.portfolio.stock_account.positions.keys())) > 0:
        for stock in list(context.portfolio.stock_account.positions.keys()):
            if stock not in stock_list:
                order_target(stock, 0)
    if len(stock_list) > 0:
        for stock in stock_list:
            if stock not in list(context.portfolio.stock_account.positions.keys()):
                if len(list(context.portfolio.stock_account.positions.keys())) < context.n :
                    number = context.n  - len(list(context.portfolio.stock_account.positions.keys()))
                    order_value(stock,context.portfolio.stock_account.available_cash/number)
                else: 
                    order_value(stock,context.portfolio.stock_account.available_cash)

                    